## Import

In [114]:
from tqdm import tqdm

import numpy as np
import pandas as pd
import re, time

* etf com crawling

In [273]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

import requests
import pickle

In [5]:
import os
os.path.sep

'\\'

In [9]:
options = webdriver.ChromeOptions()

options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.82 Safari/537.36")

driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)

<ipython-input-9-b38ee1fd7cba>:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("C:\\Users\\PC0\\Desktop\\chromedriver.exe", options=options)


* etf url per sector

In [10]:
# url 접속
driver.get("https://www.etf.com/channels/")

In [31]:
# sector 정보
sector = driver.find_elements_by_css_selector("div.container.discovery-slat")[2]

<ipython-input-31-34923382a4dc>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  sector = driver.find_elements_by_css_selector("div.container.discovery-slat")[2]


In [67]:
# sector 이름 수집
sector_names = sector.find_elements_by_css_selector("div.row > div.col-xs-12")[1:]
sector_names = [x.text for x in sector_names]

In [74]:
# sector 링크 수집
sector_links = sector.find_elements_by_css_selector("div.row > div.col-xs-12 > a")
sector_links = [x.get_attribute("href") for x in sector_links if x.get_attribute("href") != None]

In [121]:
sector_dict = {x:y for x,y in zip(sector_names, sector_links)}

In [ ]:
etf_sector_tot = []
etf_names_tot = []
etf_links_tot = []

In [152]:
start_idx = 0

In [177]:
for names, links in zip(sector_names[start_idx:], sector_links[start_idx:]):
    driver.get(links)
    time.sleep(5)
    try:
        num_etf = int(re.sub("[^0-9]","",driver.find_element_by_css_selector("span.totalEtfResults").text))
    except NoSuchElementException:
        start_idx += 1
        continue
        
    for pages in range(int(np.ceil(num_etf / 20))):

        etfs = driver.find_elements_by_css_selector("td.col_1.pad-left-10 > a.linkTickerName")
        etf_names = [x.text for x in etfs]
        etf_links = [x.get_attribute("href") for x in etfs]
        
        etf_sector_tot.extend([names for x in range(len(etf_names))])
        etf_names_tot.extend(etf_names)
        etf_links_tot.extend(etf_links)
        
        if num_etf < 20:
            break
            
        time.sleep(10)
        # page click
        try:
            driver.find_elements_by_css_selector("span.glyphicon.glyphicon-chevron-right")[0].click()
        except:
            time.sleep(3)
            driver.find_elements_by_css_selector("span.glyphicon.glyphicon-chevron-right")[0].click()
        time.sleep(7)
    start_idx += 1

<ipython-input-177-07e38a907776>:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  num_etf = int(re.sub("[^0-9]","",driver.find_element_by_css_selector("span.totalEtfResults").text))
<ipython-input-177-07e38a907776>:12: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  etfs = driver.find_elements_by_css_selector("td.col_1.pad-left-10 > a.linkTickerName")
<ipython-input-177-07e38a907776>:26: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  driver.find_elements_by_css_selector("span.glyphicon.glyphicon-chevron-right")[0].click()


In [178]:
final_df = pd.DataFrame({"sector":etf_sector_tot, "etf_names":etf_names_tot,"etf_links":etf_links_tot})

In [179]:
final_df.drop_duplicates(inplace=True)

In [180]:
final_df

,sector,etf_names,etf_links
0,Aerospace & Defense,ITA,https://www.etf.com/ITA
1,Aerospace & Defense,XAR,https://www.etf.com/XAR
2,Aerospace & Defense,PPA,https://www.etf.com/PPA
3,Aerospace & Defense,DFEN,https://www.etf.com/DFEN
4,Aerospace & Defense,SHLD,https://www.etf.com/SHLD
...,...,...,...
1877,Value,CFCV,https://www.etf.com/CFCV
1878,Value,USVT,https://www.etf.com/USVT
1879,Value,DMDV,https://www.etf.com/DMDV
1880,Value,AVSC,https://www.etf.com/AVSC


In [181]:
# final_df.to_csv("sector_df.csv", index = False)

In [ ]:
final_df.index = [x for x in range(final_df.shape[0])]

* stock ratio per etf

In [329]:
tot_stocks_name = []
tot_etfs_name = []

In [330]:
start_idx = 832

In [ ]:
for etf, url in zip(final_df.etf_names[start_idx:836], final_df.etf_links[start_idx:836]):
    driver.get(url)
    time.sleep(8)
    
    # view all click
    try:
        element = driver.find_elements_by_xpath('//*[@id="top10HoldingsAppContainer"]/div/h4/span')[0]
        driver.execute_script("arguments[0].click();", element)
        
    except:
        print("error:", etf)
        start_idx += 1
        continue
        
    time.sleep(8.5)
    
    # href 속성이 존재하는 종목 수집
    stock_names = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td.view_all_column1 > a")
    stock_names = [x.text for x in stock_names]
    
    time.sleep(0.5)
    
    # href 속성이 존재하지 않는 종목 수집
    stock_names2 = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td")
    stock_names2 = [x.text for idx, x in enumerate(stock_names2) if idx % 2 == 0]
    stock_names.extend(stock_names2)
    
    etf_names = [etf for x in range(len(stock_names))]
    
    tot_stocks_name.extend(stock_names)
    tot_etfs_name.extend(etf_names)
    
    start_idx += 1

<ipython-input-331-0c57c1c9edb5>:7: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  element = driver.find_elements_by_xpath('//*[@id="top10HoldingsAppContainer"]/div/h4/span')[0]
<ipython-input-331-0c57c1c9edb5>:18: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  stock_names = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td.view_all_column1 > a")
<ipython-input-331-0c57c1c9edb5>:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  stock_names2 = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td")


In [325]:
final_st_df = pd.DataFrame({"etf_names":tot_etfs_name, "stock_names":tot_stocks_name})
final_st_df.drop_duplicates(inplace=True)
final_st_df

,etf_names,stock_names
0,KORP,U.S. Dollar
1,KORP,Bank of America Corporation 3.419% 20-DEC-2028
2,KORP,Morgan Stanley 2.188% 28-APR-2026
3,KORP,HSBC Holdings Plc 2.099% 04-JUN-2026
4,KORP,Verizon Communications Inc. 4.329% 21-SEP-2028
...,...,...
207287,AILV,"American International Group, Inc."
207288,AILV,"S&P Global, Inc."
207289,AILV,"BlackRock, Inc."
207290,AILV,"Motorola Solutions, Inc."


In [318]:
final_st_df.to_csv("final_stock_df.csv", index =False)

In [319]:
final_st_df.tail(200)

,etf_names,stock_names
132545,TPAY,
132636,TCHI,
132840,OBOR,
132985,FTAG,
133051,SATO,
...,...,...
204490,CFCV,
204567,USVT,
204955,DMDV,
205013,AVSC,


In [320]:
stock_names = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td.view_all_column1 > a")
stock_names = [x.text for x in stock_names]

# href 속성이 존재하지 않는 종목 수집
stock_names2 = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td")
stock_names2 = [x.text for idx, x in enumerate(stock_names2) if idx % 2 == 0]
stock_names.extend(stock_names2)


<ipython-input-320-afca60dae1da>:1: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  stock_names = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td.view_all_column1 > a")
<ipython-input-320-afca60dae1da>:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  stock_names2 = driver.find_elements_by_css_selector("div.view_all_table > table > tbody > tr > td")
